In [1]:
import words
import json
import os

In [ ]:
# load the vectors

# we're using the 42 billion token, 300D, GloVe model, 
# (glove.42B.300d.zip from http://nlp.stanford.edu/projects/glove/)
# with 1.9 million words in the vocab. The original 
# vector file was parsed into a numpy array and that 
# numpy array was saved to disc. The function that does
# this parsing/loading is words.create_numpy_from_glove()
# We load that numpy array and the vocab dictionary here.

glove = GloVe()
glove.load_model("data/model/dict_glove_42b_300", "data/model/vectors_glove_42b_300.npy")

In [2]:
input_folder = "data/semgraphs"
output_folder = "data/ranked_out"



for root, dirs, files in os.walk(input_folder):
    for file in files:
        if not file.startswith("."):
            with open(os.path.join(root, file), "rU") as input:
                monthwords = json.load(input)
                top_10 = words.top_n_words(monthwords, 50)
                just_words = []
                for entry in top_10:
                    just_words.append((entry[0], len(entry[1])))

                final_out_folder = root.replace(input_folder, output_folder)

                if not os.path.isdir(final_out_folder):
                    os.makedirs(final_out_folder)

                with open(os.path.join(final_out_folder, file), "wb") as output:
                    for word in just_words:
                        output.write("{} {}\n".format(word[0], word[1]))